In [ ]:
!fusermount -u drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/seoul-bike-transit/route-finder"
%ls -la

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/seoul-bike-transit/route-finder
total 974
-rw------- 1 root root   1659 May 27 01:22 coordinate.py
drwx------ 2 root root   4096 May 27 01:22 data/
-rw------- 1 root root 932230 May 27 02:00 demo.ipynb
-rw------- 1 root root   3988 May 27 01:56 demo.py
-rw------- 1 root root     51 May 19 09:09 .env
-rw------- 1 root root      4 May 27 01:22 .gitignore
-rw------- 1 root root  39096 May 27 02:06 navigator.ipynb
-rw------- 1 root root   6096 May 27 02:06 navigator.py
drwx------ 2 root root   4096 May 27 02:06 __pycache__/
drwx------ 2 root root   4096 May 27 01:22 routes/


In [ ]:
%pip install bs4 selenium webdriver_manager python-dotenv haversine pandas tqdm pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency t

In [ ]:
from bs4 import BeautifulSoup
from navigator import KakaoRouteFinder
from IPython.display import HTML
import pandas as pd
import navigator
import coordinate
import os

In [ ]:
os.environ['WDM_LOG_LEVEL'] = '0'

org, des = ('건대입구', '쥬티클럽')

org_coord = coordinate.get_coordinate(org)
des_coord = coordinate.get_coordinate(des)

(org_x, org_y) = coordinate.transform(org_coord[::-1], 'WGS84', 'WCONGNAMUL')
(des_x, des_y) = coordinate.transform(des_coord[::-1], 'WGS84', 'WCONGNAMUL')

print(f'https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt={int(org_x)},{int(org_y)},{int(des_x)},{int(des_y)}&rt1={org}&rt2={des}')

routes = {}
routes['transit'] = KakaoRouteFinder(colab=True).find_route_by_keyword('transit', org, des, time_delta=0.5, init_time=2.0)
navigator.extract_route('transit', routes['transit'])

https://map.kakao.com/?map_type=TYPE_MAP&target=car&rt=515290,1122478,517413,1118697&rt1=건대입구&rt2=쥬티클럽


[{'info': '도보15분 | 환승없음 | 요금 900원 | 2.4km', 'time': '23분'},
 {'info': '도보15분 | 환승없음 | 요금 1,200원 | 2.4km', 'time': '22분'},
 {'info': '도보18분 | 환승1회 | 요금 1,200원 | 3.1km', 'time': '29분'},
 {'info': '도보21분 | 환승1회 | 요금 1,200원 | 2.7km', 'time': '28분'},
 {'info': '도보19분 | 환승1회 | 요금 1,200원 | 2.8km', 'time': '27분'},
 {'info': '도보19분 | 환승1회 | 요금 1,250원 | 4.7km', 'time': '32분'},
 {'info': '도보23분 | 환승1회 | 요금 1,250원 ~ 2,800원 | 3.9km', 'time': '30분'},
 {'info': '도보16분 | 환승1회 | 요금 1,200원 ~ 2,150원 | 4.2km', 'time': '36분'},
 {'info': '도보23분 | 환승없음 | 요금 2,150원 | 3.4km', 'time': '31분'},
 {'info': '도보28분 | 환승1회 | 요금 1,250원 ~ 2,300원 | 4.0km', 'time': '34분'}]

In [ ]:
from tqdm import tqdm

soup = BeautifulSoup(routes['transit'], 'html.parser')
S = set()
for route in (soup.find_all('li', {"class": "TransitRouteItem"})):
    nodes = [name.text.strip().replace(" 승차", "").replace(" 하차", "").replace(" 환승", "").replace(" 정류장", "") for name in route.find_all('a', {"data-id": "name"})]
    # print(nodes)
    S.update(nodes)
    nodes = [node.text.strip() for node in route.find_all('li', {"class": "nodeName"})]
    # print(nodes)
    S.update(nodes)

seoul_bike = {}

for lat, lng in [org_coord, des_coord]:
    try:
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        pass

for s in tqdm(S, desc="Retrieve Seoul Bike Stations: "):
    bus = coordinate.df_bus[coordinate.df_bus['정류소명'] == s]
    subway = coordinate.df_subway[coordinate.df_subway['name'] == s.replace('역', '')]
    try:
        if len(subway):
            lat, lng = subway[['lat(y)', 'lng']].iloc[0]
        elif len(bus):
            lat, lng = bus[['Y좌표', 'X좌표']].iloc[0]
        else:
            # print(s, 'N/A')
            continue
        _, bike = coordinate.get_nearest_bike(float(lat), float(lng))
        bike_id, bike_name, bike_lat, bike_lng = bike[['대여소\n번호', '보관소(대여소)명', '위도(Y)', '경도(X)']]
        congnamul = coordinate.transform((bike_lng, bike_lat), 'WGS84', 'WCONGNAMUL')
        seoul_bike[bike_id] = ( \
            bike_name, \
            bike_lat, \
            bike_lng, \
            coordinate.get_distance(org_coord, (bike_lat, bike_lng)), \
            coordinate.get_distance(des_coord, (bike_lat, bike_lng)), \
            congnamul,
        )
    except:
        # print('N/A')
        pass

rows = []
for id in seoul_bike.keys():
    name, lat, lng, org_dist, des_dist, congnamul = seoul_bike[id]
    coord = (lat, lng)
    link = f'https://map.kakao.com/?map_type=TYPE_MAP&target=transit&rt=,,{congnamul[0]},{congnamul[1]}&rt2={name}'
    rows.append([f'<a href="{link}">{name}</a>', coord, org_dist-des_dist, congnamul])

df = pd.DataFrame(rows, columns=["name", "coord", "gap_dist", "congnamul"])
HTML(df.sort_values(by="gap_dist").to_html(escape=False))

Retrieve Seoul Bike Stations: 100%|██████████| 28/28 [00:12<00:00,  2.28it/s]


,name,coord,gap_dist,congnamul
0,건대입구역 5번출구 뒤,"(37.54013824, 127.0692825)",-1.675607,"(515308.0, 1122403.0)"
9,커먼그라운드,"(37.54124832, 127.06604)",-1.669342,"(514591.0, 1122711.0)"
14,성수사거리 버스정류장 앞,"(37.54257965, 127.0633087)",-1.652556,"(513987.0, 1123080.0)"
12,건대입구역 사거리(롯데백화점),"(37.53913879, 127.0706177)",-1.364124,"(515603.0, 1122126.0)"
7,영동대교 북단,"(37.53701401, 127.0610962)",-1.123674,"(513499.0, 1121535.0)"
13,신양초교앞 교차로,"(37.5352211, 127.0683975)",-0.731529,"(515113.0, 1121039.0)"
8,이튼타워리버 2차,"(37.53520966, 127.0690002)",-0.696836,"(515246.0, 1121035.0)"
3,원일교회,"(37.53805161, 127.0765991)",-0.565948,"(516925.0, 1121825.0)"
15,광진구청 앞,"(37.53950119, 127.0830002)",-0.242071,"(518339.0, 1122229.0)"
2,자양사거리 광진아크로텔 앞,"(37.53701019, 127.0822449)",0.032455,"(518172.0, 1121537.0)"


In [ ]:
finder = KakaoRouteFinder(colab=True)
bikes = sorted(seoul_bike.items(), key=lambda item: item[1][3]-item[1][4])
bike_routes = {}
for i, srt in tqdm(enumerate(bikes), desc="Finding Bike Routes: "):
    (_, (srt_name, srt_lat, srt_lng, _, _, srt_coord)) = srt
    for j in tqdm(range(i+1, len(bikes))):
        end = bikes[j]
        (_, (end_name, end_lat, end_lng, _, _, end_coord)) = end
        route = finder.find_route_by_congnamul('bike', srt_coord, end_coord, rt1=str(i), rt2=str(j), verbose=False, init=False, time_delta=0.2)
        route_id = f'{i}-{j}'
        bike_routes[route_id] = navigator.extract_route('bike', route)
del finder

Finding Bike Routes: : 0it [00:00, ?it/s]
100%|██████████| 16/16 [00:35<00:00,  2.20s/it]
Finding Bike Routes: : 1it [00:35, 35.15s/it]
100%|██████████| 15/15 [00:32<00:00,  2.17s/it]
Finding Bike Routes: : 2it [01:07, 33.66s/it]
100%|██████████| 14/14 [00:28<00:00,  2.00s/it]
Finding Bike Routes: : 3it [01:35, 31.10s/it]
100%|██████████| 13/13 [00:29<00:00,  2.29s/it]
Finding Bike Routes: : 4it [02:05, 30.62s/it]
100%|██████████| 12/12 [00:23<00:00,  1.99s/it]
Finding Bike Routes: : 5it [02:29, 28.23s/it]
100%|██████████| 11/11 [00:24<00:00,  2.24s/it]
Finding Bike Routes: : 6it [02:54, 27.02s/it]
100%|██████████| 10/10 [00:21<00:00,  2.13s/it]
Finding Bike Routes: : 7it [03:15, 25.17s/it]
100%|██████████| 9/9 [00:16<00:00,  1.88s/it]
Finding Bike Routes: : 8it [03:32, 22.55s/it]
100%|██████████| 8/8 [00:18<00:00,  2.32s/it]
Finding Bike Routes: : 9it [03:51, 21.31s/it]
100%|██████████| 7/7 [00:16<00:00,  2.41s/it]
Finding Bike Routes: : 10it [04:08, 19.95s/it]
100%|██████████| 6/6 [0

In [ ]:
finder = KakaoRouteFinder(colab=True)

org_routes = []
des_routes = []

for i, bike in tqdm(enumerate(bikes), desc="Finding Transit Routes"):
    (_, (name, _, _, _, _, coord)) = bike
    try:
        route = finder.find_route_by_congnamul('transit', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=True, time_delta=0.5, init_time=2)
        org_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', (org_x, org_y), coord, rt1=org, rt2=str(i)+name, verbose=False, init=False, time_delta=0.5)
        org_routes.append(navigator.extract_route('walk', route))
        
    try:
        route = finder.find_route_by_congnamul('transit', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=True, time_delta=0.5, init_time=2)
        des_routes.append(navigator.extract_route('transit', route))
    except:
        route = finder.find_route_by_congnamul('walk', coord, (des_x, des_y), rt1=str(i)+name, rt2=des, verbose=False, init=False, time_delta=0.5)
        des_routes.append(navigator.extract_route('walk', route))

del finder

Finding Transit Routes: 17it [02:57, 10.47s/it]


In [ ]:
import pickle5 as pickle
filename = f'routes/{org}-{des}.pkl'

with open(filename, 'wb') as f:
    data = {'org_routes': org_routes, "des_routes": des_routes, "bikes": bikes, "bike_routes": bike_routes }
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)